En este archivo, voy a desarrollar la función que se me pide en el tercer endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_3.png "Title")

Importación de las librerías necesarias para trabajar

In [1]:
import pandas as pd

Lectura de los conjuntos de datos necesarios necesarios y breve vistazo de la cabecera de los mismos

In [2]:
user_items = pd.read_parquet("../EDA/user_items_complete.parquet")
user_items.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479


In [3]:
steam_games = pd.read_parquet("../Datasets/steam_games_complete.parquet")
steam_games.head()

,item_id,item_name,developer,genres,tags,specs,release_date,price
88310,761140,Lost Summoner Kitty,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99
88311,643980,Ironbound,Secret Level SRL,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,0.00
88312,670290,Real Pool 3D - Poolians,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,0.00
88313,767400,弹炸人2222,彼岸领域,"[Action, Adventure, Casual]","[Action, Adventure, Casual]",[Single-player],2017-12-07,0.99
88315,772540,Battle Royale Trainer,Trickjump Games Ltd,"[Action, Adventure, Simulation]","[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",2018-01-04,3.99


Con estos Dataframe, analizo lo que me piden:
- A partir del género de un juego, devuelve el usuario que acumula más horas jugadas para el género dado y una lista con la acumulación de horas jugadas por año de lanzamiento. Lo que necesito es:
    - género del juego
    - año de lanzamiento 
    - id de usuario
    - horas de juego

Veamos las columnas que tenemos disponibles en todos nuestros conjuntos de datos.

In [4]:
print(user_items.columns)
print()
print(steam_games.columns)
print()

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id'],
      dtype='object')

Index(['item_id', 'item_name', 'developer', 'genres', 'tags', 'specs',
       'release_date', 'price'],
      dtype='object')



De acuerdo a estos resultados...
Las columnas que voy a usar para este trabajo son:
- user_items: user_id, item_id, item_name (por las dudas), playtime_forever
- steam_games: item_id, item_name, genres, release_date

Motivo de selección:

- user_id: Para devolver el usuario con más horas jugadas para el género dado
- item_id: Para unir con la tabla de steam_games
- item_name: Por las dudas, posiblemente no la use, ya que no la pide la función, pero no está de más tenerla en cuenta
- playtime_forever: Para hacer el conteo de la cantidad de horas que acumula el usuario en determinado genero.
- generes: Para filtrar por genero en la tabla de juegos
- release_date: Para determinar la fecha de salida de los distintos juegos con el genero dado

In [5]:
for column in user_items.columns: # Por cada columna en el dataframe user_reviews
    if column not in ['item_id', 'item_name', 'playtime_forever',
       'user_id']: # Si la columna no es la que me sirve
        user_items.drop(column, axis=1, inplace=True) # La elimino
print(user_items.columns) # Muestro las columnas que me quedaron cómo forma de comprobar los resultados

# Repito lo mismo con el otro dataframe

for column in steam_games.columns:
    if column not in ['item_id', 'item_name', 'genres',
       'release_date']:
        steam_games.drop(column, axis=1, inplace=True)
print(steam_games.columns)


Index(['item_id', 'item_name', 'playtime_forever', 'user_id'], dtype='object')
Index(['item_id', 'item_name', 'genres', 'release_date'], dtype='object')


Hechemos un vistazo a cómo quedaron los dataframe

In [6]:
user_items

,item_id,item_name,playtime_forever,user_id
0,10,Counter-Strike,6,76561197970982479
1,20,Team Fortress Classic,0,76561197970982479
2,30,Day of Defeat,7,76561197970982479
3,40,Deathmatch Classic,0,76561197970982479
4,50,Half-Life: Opposing Force,0,76561197970982479
...,...,...,...,...
5153204,346330,BrainBread 2,0,76561198329548331
5153205,373330,All Is Dust,0,76561198329548331
5153206,388490,One Way To Die: Steam Edition,3,76561198329548331
5153207,521570,You Have 10 Seconds 2,4,76561198329548331


In [7]:
steam_games

,item_id,item_name,genres,release_date
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018-01-04
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24
88313,767400,弹炸人2222,"[Action, Adventure, Casual]",2017-12-07
88315,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018-01-04
...,...,...,...,...
120439,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018-01-04
120440,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04
120441,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04
120442,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04


Primero que nada, hago un chequeo de nulos en ambos 3 dataframes

In [8]:
user_items.isnull().sum() # Ninguno, perfecto

item_id             0
item_name           0
playtime_forever    0
user_id             0
dtype: int64

In [9]:
steam_games.isnull().sum() # Perfecto

item_id         0
item_name       0
genres          0
release_date    0
dtype: int64

Ahora que ya tengo los dataframe con las columnas que necesito, lo que me resulta más fácil de determinar son los juegos que tienen un genero dado. Para ello, utilizo el dataframe steam_games

In [10]:
for lista_generos in steam_games["genres"]:
    for genero in lista_generos:
        print(genero)
        if genero == "Casual":
            break
    break

Action
Casual


In [11]:
# La magia de pandas tiene la siguiente función
steam_games.explode("genres")

,item_id,item_name,genres,release_date
88310,761140,Lost Summoner Kitty,Action,2018-01-04
88310,761140,Lost Summoner Kitty,Casual,2018-01-04
88310,761140,Lost Summoner Kitty,Indie,2018-01-04
88310,761140,Lost Summoner Kitty,Simulation,2018-01-04
88310,761140,Lost Summoner Kitty,Strategy,2018-01-04
...,...,...,...,...
120442,610660,Russian Roads,Indie,2018-01-04
120442,610660,Russian Roads,Racing,2018-01-04
120442,610660,Russian Roads,Simulation,2018-01-04
120443,658870,EXIT 2 - Directions,Casual,2017-09-02


Primer prototipo del endpoint

In [12]:
def user_for_genre(genero):
    dataframe_filtrable_por_genero = steam_games.explode("genres")
    juegos_con_el_genero_dado = dataframe_filtrable_por_genero[dataframe_filtrable_por_genero["genres"] == genero]
    return juegos_con_el_genero_dado

In [13]:
user_for_genre("Casual") # La salida es la esperada

,item_id,item_name,genres,release_date
88310,761140,Lost Summoner Kitty,Casual,2018-01-04
88312,670290,Real Pool 3D - Poolians,Casual,2017-07-24
88313,767400,弹炸人2222,Casual,2017-12-07
88322,770380,Army of Tentacles: (Not) A Cthulhu Dating Sim:...,Casual,2018-01-04
88323,768880,Beach Rules,Casual,2018-01-04
...,...,...,...,...
120436,767590,Raining blocks,Casual,2018-01-04
120439,745400,Kebab it Up!,Casual,2018-01-04
120440,773640,Colony On Mars,Casual,2018-01-04
120441,733530,LOGistICAL: South Africa,Casual,2018-01-04


Lo siguiente que puedo hacer es determinar el año de salida de cada juego partícular a partir de la fecha de salida, y ordenar los juegos por año de salida

In [14]:
# Función para extraer el año
def extraer_año(texto):
    import re
    match = re.search(r'\b\d{4}\b', texto)
    return match.group(0) if match else None

# Función para extraer la descripción
def extraer_descripcion(texto):
    import re
    match = re.search(r'\b\d{4}\b', texto)
    if match:
        return texto.replace(match.group(0), '').strip()
    else:
        return texto

# Aplicar las funciones al DataFrame
steam_games['año_salida'] = steam_games['release_date'].apply(extraer_año)
steam_games['descripción_temporal'] = steam_games['release_date'].apply(extraer_descripcion)


In [15]:
steam_games # La salida es la esperada

,item_id,item_name,genres,release_date,año_salida,descripción_temporal
88310,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",2018-01-04,2018,-01-04
88311,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",2018-01-04,2018,-01-04
88312,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",2017-07-24,2017,-07-24
88313,767400,弹炸人2222,"[Action, Adventure, Casual]",2017-12-07,2017,-12-07
88315,772540,Battle Royale Trainer,"[Action, Adventure, Simulation]",2018-01-04,2018,-01-04
...,...,...,...,...,...,...
120439,745400,Kebab it Up!,"[Action, Adventure, Casual, Indie]",2018-01-04,2018,-01-04
120440,773640,Colony On Mars,"[Casual, Indie, Simulation, Strategy]",2018-01-04,2018,-01-04
120441,733530,LOGistICAL: South Africa,"[Casual, Indie, Strategy]",2018-01-04,2018,-01-04
120442,610660,Russian Roads,"[Indie, Racing, Simulation]",2018-01-04,2018,-01-04


Segundo prototipo del endpoint

In [16]:
def user_for_genre(genero):
    dataframe_filtrable_por_genero = steam_games.explode("genres")
    juegos_con_el_genero_dado = dataframe_filtrable_por_genero[dataframe_filtrable_por_genero["genres"] == genero]
    juegos_con_el_genero_dado = juegos_con_el_genero_dado.sort_values("año_salida")
    return juegos_con_el_genero_dado

In [17]:
juegos_de_accion = user_for_genre("Action") # La salida es la esperada
juegos_de_accion

,item_id,item_name,genres,release_date,año_salida,descripción_temporal
89687,227380,Dragon's Lair,Action,1983-06-19,1983,-06-19
89855,240340,Space Ace,Action,1984-04-29,1984,-04-29
116981,308000,Gold Rush! Classic,Action,1988-12-23,1988,-12-23
106811,569200,Space Rogue Classic,Action,1989-01-01,1989,-01-01
117149,302330,Tex Murphy: Mean Streets,Action,1989-01-01,1989,-01-01
...,...,...,...,...,...,...
120377,768580,Witchball,Action,2018-01-02,2018,-01-02
103983,579180,Ys VIII: Lacrimosa of DANA / イースVIII-Lacrimosa...,Action,2018,2018,
120439,745400,Kebab it Up!,Action,2018-01-04,2018,-01-04
102742,699570,The Legendary Player - Make Your Reputation - ...,Action,2019-12-10,2019,-12-10


Ahora que ya puedo filtrar los juegos por género y ordenarlos por el año de salida, voy a continuar con la tabla de user_items, hechemosle nuevamente un vistazo.

In [18]:
user_items.head(3)

,item_id,item_name,playtime_forever,user_id
0,10,Counter-Strike,6,76561197970982479
1,20,Team Fortress Classic,0,76561197970982479
2,30,Day of Defeat,7,76561197970982479


Yo lo que tengo en la salida del prototipo de mi función recientemente creada es:

- item_id
- item_name
- genres (columna que es redundante ya que al aplicar el filtro, todos sus valores son duplicados)
- año_salida

Las otras columnas, incluyendo la de genres, son eliminables. Lo que me conviene hacer a continuación es conservar en una variable a parte la columna con los item_id que me ofrece la función cómo salida, y a partir de esta misma variable, aplicarle un filtro / mascara al otro dataframe, al de user items. Con ello obtendría otro dataframe con los usuarios que jugaron en ese genero dado.

Tercer prototipo del endpoint

In [35]:
def user_for_genre(genero):
    steam_games_filtrable_por_genero = steam_games.explode("genres")

    juegos_con_el_genero_dado = steam_games_filtrable_por_genero[steam_games_filtrable_por_genero["genres"] == genero]

    juegos_con_el_genero_dado = juegos_con_el_genero_dado.sort_values("año_salida")

    juegos_con_el_genero_dado = juegos_con_el_genero_dado.drop(columns=["genres","release_date","descripción_temporal"])

    usuarios_que_jugaron_juegos_con_el_genero_dado = pd.merge(juegos_con_el_genero_dado, user_items, on='item_id', how='inner')

    return usuarios_que_jugaron_juegos_con_el_genero_dado

jugadores_de_accion = user_for_genre("Action")
jugadores_de_accion

,item_id,item_name_x,año_salida,item_name_y,playtime_forever,user_id
0,227380,Dragon's Lair,1983,Dragon's Lair,23,UTNerd24
1,227380,Dragon's Lair,1983,Dragon's Lair,3,Azrafael
2,227380,Dragon's Lair,1983,Dragon's Lair,47,76561198007478325
3,227380,Dragon's Lair,1983,Dragon's Lair,61,ConkerBirdy
4,227380,Dragon's Lair,1983,Dragon's Lair,53,mauledbyabear
...,...,...,...,...,...,...
2720100,302810,Divinia Chronicles: Relics of Gan-Ti,2018,Divinia Chronicles: Relics of Gan-Ti,0,mda7
2720101,302810,Divinia Chronicles: Relics of Gan-Ti,2018,Divinia Chronicles: Relics of Gan-Ti,0,DeEggMeister
2720102,302810,Divinia Chronicles: Relics of Gan-Ti,2018,Divinia Chronicles: Relics of Gan-Ti,0,sadman2007
2720103,302810,Divinia Chronicles: Relics of Gan-Ti,2018,Divinia Chronicles: Relics of Gan-Ti,0,xinyo


Bueno, ahora lo que necesito es saber la cantidad de horas totales que tiene cada usuario en ese genero para poder determinar quién tiene más horas jugadas en el género dado

In [20]:
jugadores_agrupados = jugadores_de_accion.groupby(["user_id", "año_salida"]).sum().reset_index()

In [21]:
jugadores_agrupados.head(9)

,user_id,año_salida,item_id,item_name_x,item_name_y,playtime_forever
0,--000--,2009,1250948035420550,Killing FloorSaints Row 2Defence Alliance 2Lef...,Killing FloorSaints Row 2Killing Floor Mod: De...,5329
1,--000--,2010,212800,Super Crate Box,Super Crate Box,22
2,--000--,2011,5523024240105600113400,Saints Row: The ThirdPAYDAY™ The HeistTerraria...,Saints Row: The ThirdPAYDAY: The HeistTerraria...,6522
3,--000--,2012,104700209870200210730335330204360,Super MNCBlacklight: RetributionRealm of the M...,Super Monday Night CombatBlacklight: Retributi...,109346
4,--000--,2013,233190104900230410250400,Final ExamORION: PreludeWarframeHow to Survive,Final ExamORION: PreludeWarframeHow to Survive,363
5,--000--,2014,2935402246003173602855802705502427202656303028...,Guns and RobotsDefianceDouble Action: Boogaloo...,Guns and RobotsDefianceDouble Action: Boogaloo...,3827
6,--000--,2015,417890316390273350304050333930339610,Apotheon ArenasZone-OnlineEvolve Stage 2TroveD...,Apotheon ArenasZone-OnlineEvolve Stage 2TroveD...,9860
7,--000--,2016,227940444090,Heroes &amp; GeneralsPaladins®,Heroes & GeneralsPaladins,95
8,--000--,2017,304930301520,UnturnedRobocraft,UnturnedRobocraft,4105


In [22]:
jugadores_de_accion.groupby(["user_id"]).sum().reset_index()

,user_id,item_id,item_name_x,año_salida,item_name_y,playtime_forever
0,--000--,1250948035420550212800552302424010560011340010...,Killing FloorSaints Row 2Defence Alliance 2Lef...,2009200920092009201020112011201120112012201220...,Killing FloorSaints Row 2Killing Floor Mod: De...,139469
1,--ace--,2485706201132006338073028610020436024459030339...,ToribashPortal 2The Binding of IsaacSniper Eli...,2006201120112012201220122012201420142014201420...,ToribashPortal 2The Binding of IsaacSniper Eli...,69308
2,--ionex--,2485701018055010560062022426020021073023355029...,ToribashCall of Duty®: Modern Warfare® 2Left 4...,2006200920092011201120112012201220132015201520...,ToribashCall of Duty: Modern Warfare 2Left 4 D...,38300
3,-2SV-vuLB-Kg,2833903073504720298003427099300203750205950730...,Incoming ForcesNuxCondemned: Criminal OriginsC...,2002200620062009201020112012201220122012201220...,Incoming ForcesNuxCondemned: Criminal OriginsC...,42500
4,-404PageNotFound-,2402109045602420240076701645010180550351408190...,Counter-Strike: SourceF.E.A.R.Company of Heroe...,2004200520062006200620072009200920092010201020...,Counter-Strike: SourceF.E.A.R.Company of Heroe...,116783
...,...,...,...,...,...,...
67695,zzonci,24202400,The Ship: Single PlayerThe Ship: Murder Party,20062006,The Ship Single PlayerThe Ship,0
67696,zzoptimuszz,2258403808402242601047002661502304102373102246...,Sven Co-opTeeworldsNo More Room in HellSuper M...,1999200720112012201320132014201420142014201520...,Sven Co-opTeeworldsNo More Room in HellSuper M...,74307
67697,zzydrax,2401009024240730252490237310265630273110227940...,Counter-Strike: SourceCall of Duty: World at W...,2004200820112012201320142014201420162017,Counter-Strike: SourceCall of Duty: World at W...,1194
67698,zzyfo,1024080700068808000814017410948055081903393028...,Counter-StrikeCounter-Strike: SourceCounter-St...,2000200420042006200620072008200920092009201020...,Counter-StrikeCounter-Strike: SourceCounter-St...,16512


Con esas dos líneas, ya pude encontrar las dos cosas que me pedían, por un lado, con la última línea (jugadores_de_accion.groupby(["user_id"]).sum().reset_index()) puedo determinar quién es el jugador de acción que más horas jugó, lo muestro a continuación.

In [27]:
jugador_de_accion_mas_horas = jugadores_de_accion.groupby(["user_id"]).sum().reset_index()
jugador_de_accion_mas_horas[jugador_de_accion_mas_horas["playtime_forever"] == jugador_de_accion_mas_horas["playtime_forever"].max()]

,user_id,item_id,item_name_x,año_salida,item_name_y,playtime_forever
50550,Sp3ctre,2834303063502923907020501069001302053068502833...,Litil DivilSlipstream 5000Realms of the Haunti...,1993199519961998199919992000200020012001200220...,Litil DivilSlipstream 5000Realms of the Haunti...,1635566


Por la salida podemos determinar que "Sp3ctre" es el jugador con más horas invertidas en juegos de acción, con un total de 1635566 horas en este genero. Hagamos una salida normalizada ahora con estos descubrimientos.

Cuarto prototipo del endpoint

In [33]:
def user_for_genre(genero): # Se ingresa el genero de vídeojuegos

    steam_games_filtrable_por_genero = steam_games.explode("genres") # Se crea un nuevo dataframe a partir del original de vídeojuegos con el método .explode, lo cuál facilita que sea filtrado posteriormente por el genero ingresado.

    juegos_con_el_genero_dado = steam_games_filtrable_por_genero[steam_games_filtrable_por_genero["genres"] == genero] # Se filtra el dataframe previamente creado por el genero ingresado en la función

    juegos_con_el_genero_dado = juegos_con_el_genero_dado.drop(columns=["genres","release_date","descripción_temporal"]) # Se eliminan las columnas irrelevantes para optimizar el rendimiento de la función. 

    juegos_con_el_genero_dado = juegos_con_el_genero_dado.sort_values("año_salida") # Se ordenan los juegos por el año de salida

    usuarios_que_jugaron_juegos_con_el_genero_dado = pd.merge(juegos_con_el_genero_dado, user_items, on='item_id', how='inner') # Se unen los datafrmae de juegos con el genero ingresado con la función (juegos_con_el_genero_dado), con el de juegos que tienen los usuarios (user_items).

    usuarios_del_genero_con_horas_sumadas = usuarios_que_jugaron_juegos_con_el_genero_dado.groupby(["user_id"]).sum().reset_index() # Se suman las horas de cada usuario de todos los juegos que jugó. EJ: Juan jugó dos juegos, 2 y 3 horas respectivamente, en total Juan jugó 5 horas.

    jugador_del_genero_con_mas_horas = usuarios_del_genero_con_horas_sumadas[usuarios_del_genero_con_horas_sumadas["playtime_forever"] == usuarios_del_genero_con_horas_sumadas["playtime_forever"].max()]["user_id"] # Se filtra el dataframe por el usuario que más horas acumule en el genero dado, y se conserva sólo el user_id

    return jugador_del_genero_con_mas_horas

user_for_genre("Action") # La salida es la esperada

50550    Sp3ctre
Name: user_id, dtype: object

Probemos con otro género para confirmar.

In [34]:
user_for_genre("Simulation") # clawbot44

46562    clawbot44
Name: user_id, dtype: object

In [37]:
# Volví a ejecutar el tercer prototipo del endpoint, y luego
jugadores_de_simulacion = user_for_genre("Simulation")
jugadores_de_simulacion

,item_id,item_name_x,año_salida,item_name_y,playtime_forever,user_id
0,374080,Falcon,1987,Falcon,0,phrostb
1,374080,Falcon,1987,Falcon,0,mayshowganmore
2,374080,Falcon,1987,Falcon,0,76561197983589335
3,374080,Falcon,1987,Falcon,0,djnemonic
4,374080,Falcon,1987,Falcon,0,DownSyndromeKid
...,...,...,...,...,...,...
525112,287220,Autocraft,2017,Autocraft,22,76561198073245886
525113,287220,Autocraft,2017,Autocraft,3,76561198074822440
525114,287220,Autocraft,2017,Autocraft,148,76561198080801224
525115,287220,Autocraft,2017,Autocraft,163,AzerTheGreat


La salida del siguiente código debéria corresponder a la fila del jugador clawbot44, veamos...

In [38]:
jugador_de_simulacion_horas_totales = jugadores_de_simulacion.groupby(["user_id"]).sum().reset_index()
jugador_de_simulacion_horas_totales[jugador_de_simulacion_horas_totales["playtime_forever"] == jugador_de_simulacion_horas_totales["playtime_forever"].max()] # La salida es la esperada

,user_id,item_id,item_name_x,año_salida,item_name_y,playtime_forever
46562,clawbot44,4000400031416031416025034025034017390173903393...,Garry's ModGarry's ModMicrosoft Flight Simulat...,2006200620062006200720072008200820102010201120...,Garry's ModGarry's ModMicrosoft Flight Simulat...,798258


Por otro lado, y con la primera línea (jugadores_de_accion.groupby(["user_id", "año_salida"]).sum().reset_index()), obtuve otra de las cosas que me pedían, o más bien estoy cerca de obtenerlo, que sería, de acuerdo al año de salida de cada juego, las horas acumuladas por el usuario en dicho año.

Veamos de vuelta ese código, esta vez con los jugadroes de simulación

In [39]:
jugadores_simulacion_agrupados = jugadores_de_simulacion.groupby(["user_id", "año_salida"]).sum().reset_index()
jugadores_simulacion_agrupados

,user_id,año_salida,item_id,item_name_x,item_name_y,playtime_forever
0,--000--,2006,4000,Garry's Mod,Garry's Mod,925
1,--000--,2014,263760305260,Turbo Dismount™Battle Islands,Turbo DismountBattle Islands,244
2,--000--,2015,363970,Clicker Heroes,Clicker Heroes,11
3,--000--,2017,301520,Robocraft,Robocraft,68
4,--ace--,2006,4000,Garry's Mod,Garry's Mod,80
...,...,...,...,...,...,...
275973,zzyfo,2010,33930,Arma 2: Operation Arrowhead,Arma 2: Operation Arrowhead,8
275974,zzyfo,2012,212680,FTL: Faster Than Light,FTL: Faster Than Light,84
275975,zzyfo,2013,224580239820,Arma II: DayZ ModGame Dev Tycoon,Arma 2: DayZ ModGame Dev Tycoon,2056
275976,zzzmidmiss,2014,253710,theHunter Classic,theHunter,16


El problema con el código anterior es que agruparlos por user_id y año_salida no es lo más correcto, ya que por ejemplo, dos juegos que jugó el usuario pudieron ser lanzados en el mismo año, cómo sucede en la fila 2 y la 275976.
 Más allá de que no sea lo más correcto debido a la superposición de los nombres cómo muestran esas dos filas, para el desarrollo de la función que me piden, sirve, ya que de la misma manera que se superponen los nombres, las horas también se "superponen", pero al ser números se suman, entonces, ya tengo la suma por año. Hagamos algunas prubas más.

Primero, filtremos el dataframe por el usuario con más horas, en el caso de simulation ya sabemos que es clawbot44, entonces:

In [42]:
jugador_top_distintos_años = jugadores_simulacion_agrupados[jugadores_simulacion_agrupados["user_id"] == "clawbot44"]
jugador_top_distintos_años

,user_id,año_salida,item_id,item_name_x,item_name_y,playtime_forever
213524,clawbot44,2006,40004000314160314160,Garry's ModGarry's ModMicrosoft Flight Simulat...,Garry's ModGarry's ModMicrosoft Flight Simulat...,723162
213525,clawbot44,2007,250340250340,BlocklandBlockland,BlocklandBlockland,1064
213526,clawbot44,2008,1739017390,SPORE™SPORE™,SporeSpore,600
213527,clawbot44,2010,3393033930,Arma 2: Operation ArrowheadArma 2: Operation A...,Arma 2: Operation ArrowheadArma 2: Operation A...,404
213528,clawbot44,2011,6573065730,Take On HelicoptersTake On Helicopters,Take On HelicoptersTake On Helicopters,120
213529,clawbot44,2013,2245802245802273002273002341902341901074101074...,Arma II: DayZ ModArma II: DayZ ModEuro Truck S...,Arma 2: DayZ ModArma 2: DayZ ModEuro Truck Sim...,8956
213530,clawbot44,2014,2659302659302537102537102632802632802429202429...,Goat SimulatorGoat SimulatortheHunter Classict...,Goat SimulatorGoat SimulatortheHuntertheHunter...,7440
213531,clawbot44,2015,3278903278902202002202003339503339503895703895...,I am BreadI am BreadKerbal Space ProgramKerbal...,I am BreadI am BreadKerbal Space ProgramKerbal...,49434
213532,clawbot44,2016,324310324310237870237870,Bus Simulator 16Bus Simulator 16Planet Explore...,Bus Simulator 16Bus Simulator 16Planet Explore...,792
213533,clawbot44,2017,244030244030305620305620301520301520,Take On MarsTake On MarsThe Long DarkThe Long ...,Take On MarsTake On MarsThe Long DarkThe Long ...,6286


Ahora voy a desarrollar un bucle que recorra el dataframe jugador_top recientemente creado. Con ello crearé el diccionario de la forma {2001:20 horas, 2002: 35 horas ...}

In [48]:
for año in jugador_top_distintos_años["año_salida"]: # Bucle que recorre cada año del dataframe
    # A partir de los años, queremos saber el valor correspondiente a playtime_forever
    print(jugador_top_distintos_años[jugador_top_distintos_años["año_salida"] == año]["playtime_forever"])


213524    723162
Name: playtime_forever, dtype: int64
213525    1064
Name: playtime_forever, dtype: int64
213526    600
Name: playtime_forever, dtype: int64
213527    404
Name: playtime_forever, dtype: int64
213528    120
Name: playtime_forever, dtype: int64
213529    8956
Name: playtime_forever, dtype: int64
213530    7440
Name: playtime_forever, dtype: int64
213531    49434
Name: playtime_forever, dtype: int64
213532    792
Name: playtime_forever, dtype: int64
213533    6286
Name: playtime_forever, dtype: int64


In [55]:
horas_jugadas_por_año = {}
for año in jugador_top_distintos_años["año_salida"]: # Bucle que recorre cada año del dataframe
    # A partir de los años, queremos saber el valor correspondiente a playtime_forever
    playtime = jugador_top_distintos_años[jugador_top_distintos_años["año_salida"] == año]["playtime_forever"]
    horas_jugadas_por_año[int(año)] = int(playtime.iloc[0])
    

print(horas_jugadas_por_año)


{2006: 723162, 2007: 1064, 2008: 600, 2010: 404, 2011: 120, 2013: 8956, 2014: 7440, 2015: 49434, 2016: 792, 2017: 6286}


Añadamos esto a nuestro prototipo de endpoint 3

Quinto prototipo del endpoint

In [76]:
def user_for_genre(genero): # Se ingresa el genero de vídeojuegos

    diccionario_de_salida = {} # Se crea el diccionario que posteriormente va a devolver la función cómo salida

    steam_games_filtrable_por_genero = steam_games.explode("genres") # Se crea un nuevo dataframe a partir del original de vídeojuegos con el método .explode, lo cuál facilita que sea filtrado posteriormente por el genero ingresado.

    juegos_con_el_genero_dado = steam_games_filtrable_por_genero[steam_games_filtrable_por_genero["genres"] == genero] # Se filtra el dataframe previamente creado por el genero ingresado en la función

    juegos_con_el_genero_dado = juegos_con_el_genero_dado.drop(columns=["genres","release_date","descripción_temporal"]) # Se eliminan las columnas irrelevantes para optimizar el rendimiento de la función. 

    juegos_con_el_genero_dado = juegos_con_el_genero_dado.sort_values("año_salida") # Se ordenan los juegos por el año de salida

    usuarios_que_jugaron_juegos_con_el_genero_dado = pd.merge(juegos_con_el_genero_dado, user_items, on='item_id', how='inner') # Se unen los datafrmae de juegos con el genero ingresado con la función (juegos_con_el_genero_dado), con el de juegos que tienen los usuarios (user_items).

    usuarios_que_jugaron_juegos_con_el_genero_dado.drop(columns=["item_id","item_name_y"], inplace=True) # Nuevamente se eliminan las columnas irrelevante para optimizar el rendimiento de la función

    usuarios_del_genero_con_horas_sumadas = usuarios_que_jugaron_juegos_con_el_genero_dado.groupby(["user_id"]).sum().reset_index() # Se suman las horas de cada usuario de todos los juegos que jugó. EJ: Juan jugó dos juegos, 2 y 3 horas respectivamente, en total Juan jugó 5 horas.

    jugador_del_genero_con_mas_horas = usuarios_del_genero_con_horas_sumadas[usuarios_del_genero_con_horas_sumadas["playtime_forever"] == usuarios_del_genero_con_horas_sumadas["playtime_forever"].max()]["user_id"].iloc[0] # Se filtra el dataframe por el usuario que más horas acumule en el genero dado, y se conserva sólo el user_id

    diccionario_de_salida[f"Usuario con más horas para el género {genero}"] = jugador_del_genero_con_mas_horas

    datos_usuario_top = usuarios_que_jugaron_juegos_con_el_genero_dado[(usuarios_que_jugaron_juegos_con_el_genero_dado["user_id"] == jugador_del_genero_con_mas_horas) & (usuarios_que_jugaron_juegos_con_el_genero_dado["playtime_forever"] != 0)]

    jugador_top_del_genero_agrupado_por_nombre_y_año = datos_usuario_top.groupby([ "año_salida"]).sum().reset_index()
    horas_jugadas_por_año = []
    for año in jugador_top_distintos_años["año_salida"]: # Bucle que recorre cada año del dataframe
        # A partir de los años, queremos saber el valor correspondiente a playtime_forever
        tiempo_de_juego_de_cada_año = jugador_top_distintos_años[jugador_top_distintos_años["año_salida"] == año]["playtime_forever"]
        diccionario_temporal = {}
        diccionario_temporal[int(año)] = int(tiempo_de_juego_de_cada_año.iloc[0])
        horas_jugadas_por_año.append(diccionario_temporal)
        

    diccionario_de_salida["Horas jugadas en los distintos años"] = horas_jugadas_por_año

    return diccionario_de_salida

user_for_genre("Action") # La salida es la esperada

{'Usuario con más horas para el género Action': 'Sp3ctre',
 'Horas jugadas en los distintos años': [{2006: 723162},
  {2007: 1064},
  {2008: 600},
  {2010: 404},
  {2011: 120},
  {2013: 8956},
  {2014: 7440},
  {2015: 49434},
  {2016: 792},
  {2017: 6286}]}